In [ ]:
# Qwipo Prototype Notebook
import pandas as pd
from src.features import load_data, build_id_maps, build_item_user_matrix
from src.cf_model import CFModelSVD
from src.cb_model import CBModel
from src.hybrid import HybridRecommender

products_df, transactions_df = load_data()
u2idx, i2idx = build_id_maps(transactions_df)
item_user = build_item_user_matrix(transactions_df, u2idx, i2idx)

cf = CFModelSVD(n_factors=20)
cf.fit(item_user)

cb = CBModel()
cb.build_embeddings(products_df)

i2idx_map = i2idx
p_index_map = {pid: idx for idx, pid in enumerate(products_df['product_id'].tolist())}

hybrid = HybridRecommender(cf, cb, i2idx_map, p_index_map)

# Test recommendation
retailer_id = list(u2idx.keys())[0]
user_idx = u2idx[retailer_id]
recs = hybrid.recommend(user_idx, item_user.T.tocsr(), top_k=5)
for idx, score in recs:
    print(idx, score)
